In [2]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source("cleanup.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [20]:
diabetes_df <- read_csv("/home/jupyter/Group41/data/diabetes.csv") %>% 
    mutate(Outcome = as_factor(Outcome))

diabetes_df 

Parsed with column specification:
cols(
  Pregnancies = col_double(),
  Glucose = col_double(),
  BloodPressure = col_double(),
  SkinThickness = col_double(),
  Insulin = col_double(),
  BMI = col_double(),
  DiabetesPedigreeFunction = col_double(),
  Age = col_double(),
  Outcome = col_double()
)



Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
5,121,72,23,112,26.2,0.245,30,0
1,126,60,0,0,30.1,0.349,47,1
1,93,70,31,0,30.4,0.315,23,0


In [51]:
set.seed(12398)

diabetes_split <- initial_split(diabetes_df, prop = 0.75, strata = Outcome)
diabetes_training <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

diabetes_test

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0,137,40,35,168,43.1,2.288,33,1
10,168,74,0,0,38.0,0.537,34,1
13,145,82,19,110,22.2,0.245,57,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
11,120,80,37,150,42.3,0.785,48,1
4,136,70,0,0,31.2,1.182,22,1
5,121,72,23,112,26.2,0.245,30,0


In [52]:
diabetes_count <- diabetes_training %>%
    group_by(Outcome) %>%
    count(Outcome)

diabetes_count

Outcome,n
<fct>,<int>
0,375
1,201


In [56]:
set.seed(2020)

diabetes_count <- diabetes_training %>%
    group_by(Outcome) %>%
    count(Outcome)

diabetes_training_recipe <- recipe(Outcome ~ Pregnancies + Glucose + BloodPressure + Insulin + BMI + DiabetesPedigreeFunction + Age, 
                                   data = diabetes_training) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

diabetes_train_fold <- vfold_cv(diabetes_training, v = 5, strata = Outcome)

diabetes_training_knn <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>% 
    set_engine("kknn") %>% 
    set_mode("classification")

diabetes_train_w <- workflow() %>%
    add_recipe(diabetes_training_recipe) %>% 
    add_model(diabetes_training_knn) %>% 
    #fit_resample(resample = diabetes_train_vfold) 
    tune_grid(resamples = diabetes_train_fold, grid =10) %>%
    collect_metrics()

diabetes_train_w

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.7030885,5,0.02040003,Model1
1,roc_auc,binary,0.6611585,5,0.02032958,Model1
4,accuracy,binary,0.7291604,5,0.01813247,Model2
⋮,⋮,⋮,⋮,⋮,⋮,⋮
13,roc_auc,binary,0.7999618,5,0.02482711,Model8
15,accuracy,binary,0.7447676,5,0.01027458,Model9
15,roc_auc,binary,0.8073959,5,0.02417428,Model9
